# Connect to a Database

In [3]:
import os
from configparser import ConfigParser
from mysql.connector import MySQLConnection

### Bad practice!

In [4]:
conn = MySQLConnection(host='localhost',
                       database='school',
                       user='root',
                       password='seekrit')

In [5]:
conn

In [6]:
conn.close()

### Instead, use a configuration file.

```
[mysql]
host = localhost
database = school
user = root
password = seekrit
```
**`school.ini`**

### Use the configuration parser

In [7]:
def read_config(config_file = 'config.ini', section = 'mysql'):
    """
    Read a configuration file config_file and the given section. 
    If successful, return the configuration as a dictionary,
    else raise an exception. 
    """
    parser = ConfigParser()
    
    # Does the configuration file exist?
    if os.path.isfile(config_file):
        parser.read(config_file)
    else:
        raise Exception(f"Configuration file '{config_file}' "
                        "doesn't exist.")
    
    config = {}
    
    # Does it have the right section?
    if parser.has_section(section):
        
        # Parse the configuration file.
        items = parser.items(section)
        
        # Construct the parameter dictionary.
        for item in items:
            config[item[0]] = item[1]
            
    else:
        raise Exception(f"Section '{section}' missing "
                        f"in configuration file '{config_file}'.")
    
    return config

In [8]:
db_config = read_config('school.ini')
db_config

{'host': 'localhost',
 'database': 'school',
 'user': 'root',
 'password': 'seekrit'}

### Then the connection code is simpler and doesn’t expose the password.

In [9]:
def make_connection(config_file = 'config.ini', section = 'mysql'):
    """
    Make a connection to a database with the configuration file
    config_file and the given section. If successful, return 
    the connection, else raise an exception.
    """
    try:
        db_config = read_config(config_file, section)            
        conn = MySQLConnection(**db_config)
        
        if conn.is_connected():
            return conn
                
    except Error as e:
        raise Exception(f'Connection failed.\n{e}')

In [10]:
conn = make_connection('school.ini')
conn

In [11]:
cursor = conn.cursor()
cursor

In [12]:
sql = "SELECT * FROM class"

cursor.execute(sql)
rows  = cursor.fetchall()
count = cursor.rowcount

print(f'Fetched {count} rows.')
print()

for row in rows:
    print(row)

Fetched 5 rows.

('DATA 200', 'Python Programming', 101, 'T7051')
('DATA 220', 'Mathematical Models', 114, 'T7008')
('DATA 225', 'Database Systems', 101, 'T7003')
('DATA 240', 'Data Mining', 120, 'T7012')
('DATA 245', 'Machine Learning', 115, 'T7051')


In [13]:
cursor.close()
conn.close()

### It will be easy to switch to another database, even on a different server.

```
[mysql]
host = IES-ADS-ClassDB.sjsu.edu
database = school
user = student
password = seekrit
```

**`school-remote.ini`**

In [14]:
conn = make_connection('school-remote.ini')
conn

NameError: name 'Error' is not defined

In [15]:
cursor = conn.cursor()
cursor

OperationalError: MySQL Connection not available

In [15]:
sql = "SELECT * FROM student"

cursor.execute(sql)
rows  = cursor.fetchall()
count = cursor.rowcount

print(f'Fetched {count} rows.')
print()

for row in rows:
    print(row)

Fetched 5 rows.

('S1001', 'Doe', 'John', 'C03')
('S1005', 'Nova', 'Tim', 'C04')
('S1009', 'Klein', 'Leslie', 'C05')
('S1014', 'Jane', 'Mary', 'C01')
('S1021', 'Smith', 'Kim', 'C02')


In [16]:
cursor.close()
conn.close()

### Using a configuration file to make a database connection is code we’ll use often, so let’s put it into a separate Python database utilities file **`DATA225utils.py`** containing the two functions. We can then import the file whenever necessary.

In [ ]:
# Copyright (c) 2023 by Ronald Mak